In [ ]:
%pip install pyalgotrade
%pip install pandas

In [84]:
import numpy as np
import pandas as pd

from pyalgotrade import strategy, barfeed, bar, plotter
from pyalgotrade.bar import Frequency
from pyalgotrade.technical import ma, bollinger
from pyalgotrade.stratanalyzer import returns, sharpe

In [89]:
df = pd.read_csv("NIFTY50.csv")

cols = df.columns[1:] 
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df['Date'] = pd.to_datetime(df['Date'])
df['Volume'] = np.iinfo(np.int32).max  

df.head()

,Date,Open,High,Low,Close,Volume
0,2012-12-03,5878.25,5899.15,5854.60,5870.9,2147483647
1,2012-12-04,5866.80,5894.95,5859.00,5889.2,2147483647
2,2012-12-05,5906.60,5917.80,5891.35,5900.5,2147483647
3,2012-12-06,5926.30,5942.55,5838.90,5930.9,2147483647
4,2012-12-07,5934.00,5949.85,5888.65,5907.4,2147483647


In [90]:
# Example BarFeed for dataframes with data for a single instrument.
class DataFrameBarFeed(barfeed.BaseBarFeed):
    def __init__(self, dataframe, instrument, frequency):
        super(DataFrameBarFeed, self).__init__(frequency)
        self.registerInstrument(instrument)
        self.__df = dataframe
        self.__instrument = instrument
        self.__next = 0

    def reset(self):
        super(DataFrameBarFeed, self).reset()
        self.__next = 0

    def peekDateTime(self):
        return self.getCurrentDateTime()

    def getCurrentDateTime(self):
        ret = None
        if not self.eof():
            rowkey = self.__df.index[self.__next]
            ret = df.iloc[rowkey]["Date"]
        return ret

    def barsHaveAdjClose(self):
        return False

    def getNextBars(self):
        ret = None
        if not self.eof():
            # Convert the dataframe row into a bar.BasicBar
            rowkey = self.__df.index[self.__next]
            row = self.__df.iloc[rowkey]
            bar_dict = {
                self.__instrument: bar.BasicBar(
                    row["Date"],
                    row["Open"],
                    row["High"],
                    row["Low"],
                    row["Close"],
                    row["Volume"],
                    None,  # row["Adj Close"],
                    self.getFrequency(),
                )
            }

            ret = bar.Bars(bar_dict)
            self.__next += 1
        return ret

    def eof(self):
        return self.__next >= len(self.__df.index)

    def start(self):
        pass

    def stop(self):
        pass

    def join(self):
        pass


In [ ]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, capital):
        super(MyStrategy, self).__init__(feed, capital)
        self.__instrument = instrument
        self.__position = None
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)

    @staticmethod
    def safe_round(value, digits):
        if value is not None:
            value = round(value, digits)
        return value

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at \u20B9 %.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at \u20B9 %.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        self.__position.exitMarket()

    def onBars(self, bars):
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                self.__position = self.enterLong(self.__instrument, 10, True)
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


capital = 100000
feed = DataFrameBarFeed(df, "nifty", Frequency.DAY)
myStrategy = MyStrategy(feed, "nifty", capital)

returnsAnalyzer = returns.Returns()
myStrategy.attachAnalyzer(returnsAnalyzer)

plt = plotter.StrategyPlotter(myStrategy)

plt.getOrCreateSubplot("returns").addDataSeries("Simple returns", returnsAnalyzer.getReturns())


myStrategy.run()
myStrategy.info("Final portfolio value: $%.2f" % myStrategy.getResult())

# Plot the strategy.
plt.plot()

In [ ]:
from pyalgotrade import strategy
from pyalgotrade.technical import bollinger
from pyalgotrade.stratanalyzer import sharpe

# Work in Progress
class BBands(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, bBandsPeriod, capital):
        super(BBands, self).__init__(feed, capital)
        self.__instrument = instrument
        self.__bbands = bollinger.BollingerBands(
            feed[instrument].getCloseDataSeries(), bBandsPeriod, 1.5
        )
        self.__position = None

    def getBollingerBands(self):
        return self.__bbands

    def onEnterCanceled(self, position):
        #restore position
        self.__position = None

    def onEnterOK(self, position):
        print(position)
        pass

    def onExitOk(self, position):
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        lower = self.__bbands.getLowerBand()[-1]
        upper = self.__bbands.getUpperBand()[-1]
        if lower is None:
            return

        shares = self.getBroker().getShares(self.__instrument)
        bar = bars[self.__instrument]

        if self.__position is None and bar.getHigh() < lower:
            sharesToBuy = int(self.getBroker().getCash(False) / bar.getHigh())
            self.info("Placing Buy Stop order")
            self.stopOrder(self.__instrument, bar.getHigh(), sharesToBuy)
            # update SL, TL , engagement in self.__position

        elif self.__position.isActive() and (bar.getClose() <= self.__position.SL or bar.getClose() >= self.__position.TL):
            self.info("Placing Sell Market order")
            self.marketOrder(self.__instrument, -self.__position.shares)


instrument = "nifty"
bBandsPeriod = 40
capital = 100000

strat = BBands(feed, instrument, bBandsPeriod, capital)
sharpeRatioAnalyzer = sharpe.SharpeRatio()
strat.attachAnalyzer(sharpeRatioAnalyzer)

plt = plotter.StrategyPlotter(strat, True, True, True)
plt.getInstrumentSubplot(instrument).addDataSeries("upper", strat.getBollingerBands().getUpperBand())
plt.getInstrumentSubplot(instrument).addDataSeries("lower", strat.getBollingerBands().getLowerBand())

strat.run()
print("Sharpe ratio: %.2f" % sharpeRatioAnalyzer.getSharpeRatio(0.05))

plt.plot()


To create a strategy that uses Bollinger Bands and places market and limit orders in PyAlgoTrade, you can use the following steps:

1. Create a class that extends the `strategy.BacktestingStrategy` class. This class will contain the logic for your trading strategy.
2. In the `__init__` method of your `strategy` class, initialize the Bollinger Bands indicator and a `broker.Broker` object. You can use the indicator.BollingerBands class to create the Bollinger Bands indicator, and the `broker.backtesting`.Broker class to create a `broker.Broker` object for backtesting.
3. Override the `onBars` method of the `strategy` class. This method is called for each bar in the data set and contains the logic for your trading strategy.
4. In the onBars method, use the Bollinger Bands indicator to calculate the upper and lower bands for the current bar. Then, use the `broker.Broker` object to place market and limit orders based on the position of the security relative to the Bollinger Bands. You can use the `broker.placeMarketOrder` and `broker.createLimitOrder` methods to place market and limit orders, respectively.
5. To track the position of your investment, you can use the `position.Position` class. In the `onBars` method, create a `position.Position` object for the security and use the update method to update the position with each new trade.